In [1]:
import os, sys
import numpy as np
import dask
import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox.utils import *

/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [2]:
from dask_jobqueue import PBSCluster
cluster = PBSCluster()
w = cluster.start_workers(20)

/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils.py:128: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  % (host, default, e), RuntimeWarning)


In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://10.148.1.19:54983 Dashboard: http://10.148.1.19:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


---

# store grid

In [5]:
ds_index, ds = get_compressed_level_index(grid_dir)
#print(ds)

ds = ds.drop(['dxG','dyG','dxC','dyC','rAw','rAs','rAz'])
ds = ds.drop(['hFacC','hFacW','hFacS'])
ds = ds.drop(['Z', 'Zp1', 'Zu', 'Zl', 'drC', 'drF','PHrefC','PHrefF'])

#Nc = 432 # original choice
#Nc = 27 # very long scheduling
Nc = 96
ds = ds.chunk({'i': Nc, 'j': Nc, 'i_g': Nc, 'j_g': Nc})

print(ds)

/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:314: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, i_g: 4320, j: 4320, j_g: 4320, k: 90, k_l: 90, k_p1: 91, k_u: 90)
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j_g      (j_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * k_u      (k_u) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k_l      (k_l) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k_p1     (k_p1) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
    XC       (face, j, i) >f4 dask.array<shape=(13, 4320, 4320), chunksize=(1, 96, 96)>
    YC       (face, j, i) >f4 dask.array<shape=(13, 4320, 4320), chunksize=(1, 96, 96)>
    

In [10]:
out_dir = scratch+'/'
#out_dir = osi

In [11]:
file_out = out_dir+'mit_grid.zarr'
%time ds.to_zarr(file_out, mode='w')

In [12]:
dst = ds.transpose('face', 'i', 'j', 'i_g', 'j_g', 'k', 'k_l', 'k_p1', 'k_u')
print(dst)

<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, i_g: 4320, j: 4320, j_g: 4320, k: 90, k_l: 90, k_p1: 91, k_u: 90)
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j_g      (j_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * k_u      (k_u) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k_l      (k_l) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k_p1     (k_p1) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
    XC       (face, i, j) >f4 dask.array<shape=(13, 4320, 4320), chunksize=(1, 96, 96)>
    YC       (face, i, j) >f4 dask.array<shape=(13, 4320, 4320), chunksize=(1, 96, 96)>
    

In [13]:
file_out = out_dir+'mit_grid_t.zarr'
%time dst.to_zarr(file_out, mode='w')

CPU times: user 4min 15s, sys: 6.49 s, total: 4min 21s
Wall time: 4min 30s


--- 

# automatic rewriting of all variables

# transposed data: (i,j,time), 1 file per face, time windows

Not working at the moment, not enough memory

In [20]:
V = ['Eta', 'SST', 'SSS', 'SSU', 'SSV']
V = ['SSU']

Nt = 24*10 # time chunks
#Nt = 0

#Nc = 432 # original choice
Nc = 96 #
#Nc = 27 # very long scheduling

out_dir = scratch+'/mit_zarr_t/'
#out_dir = datawork+'/mit_zarr_t/'
#out_dir = osi+'mit_zarr_t/'

for v in V:
    #
    data_dir = root_data_dir+v+'/'
    iters, time = get_iters_time(v, data_dir, delta_t=25.)
    #
    p = 'C'
    if v is 'SSU':
        p = 'W'
    elif v is 'SSV':
        p = 'S'
    #
    ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
    #
    lNt = len(ds.time) if Nt == 0 else Nt
    # tmp
    #ds = ds.isel(face=slice(1,3))
    ds = ds.isel(face=1) # seems to fix issue with memory blow up
    # maybe chunk along face here
    #
    dv = ds[v].isel(time=slice(len(ds.time)//lNt *lNt))
    print(dv)
    #
    dv = dv.drop(['XC','YC','Depth','rA']).to_dataset()
    #
    #dv = dv.chunk({'time': lNt})
    dv = dv.chunk({'time': lNt, 'i': Nc, 'j': Nc})
    #dv = dv.chunk({'face': 1, 'time': lNt, 'i': Nc, 'j': Nc})
    #
    #dv = dv.transpose('face','i','j','time') # commented tmp, still leads to memory blow up
    #
    file_out = out_dir+'%s.zarr'%(v)
    try:
        print(dv)
        %time dv.to_zarr(file_out, mode='w')
    except:
        print('Failure')
    dsize = getsize(file_out)
    print('   data is %.1fGB ' %(dsize/1e9))


/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:314: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


<xarray.DataArray 'SSU' (time: 8640, j: 4320, i: 4320)>
dask.array<shape=(8640, 4320, 4320), dtype=>f4, chunksize=(1, 4320, 4320)>
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
    face     int64 1
    XC       (j, i) >f4 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    YC       (j, i) >f4 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    rA       (j, i) >f4 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
    Depth    (j, i) >f4 dask.array<shape=(4320, 4320), chunksize=(4320, 4320)>
  * time     (time) float64 5.702e+06 5.706e+06 5.71e+06 5.713e+06 5.717e+06 ...
    dtime    (time) datetime64[ns] 2011-11-15 2011-11-15T01:00:00 ...
<xarray.Dataset>
Dimensions:  (i: 4320, j: 4320, time: 8640)
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13

KeyboardInterrupt: 

   data is 176.9GB 


In [18]:
V = ['Eta', 'SST', 'SSS', 'SSU', 'SSV']
V = ['SSU']

Nt = 24*10 # time chunks
Nt = 0
out_dir = scratch+'/mit_zarr_t/'
#out_dir = datawork+'/mit_zarr_t/'
#out_dir = osi+'mit_zarr_t/'

for v in V:
    #
    data_dir = root_data_dir+v+'/'
    iters, time = get_iters_time(v, data_dir, delta_t=25.)
    #
    p = 'C'
    if v is 'SSU':
        p = 'W'
    elif v is 'SSV':
        p = 'S'
    #
    ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
    #
    if Nt == 0:
        Nt = len(ds.time)
    #
    #for face in range(ds['face'].size):
    for face in [1]:
        file_out = out_dir+'%s_f%02d.zarr'%(v,face)
        if not os.path.isfile(file_out) or getsize(file_out) < fsize_bound:
            #dv = ds[v].isel(face=face) # does not work, incompatible zarr chunks
            dv = ds[v].isel(face=face, time=slice(len(ds.time)//Nt *Nt))
            # should store grid data independantly in a single file
            dv = dv.drop(['XC','YC','Depth','rA']).to_dataset()
            #
            dv = dv.chunk({'time': Nt, 'i': 432, 'j': 432})
            dv = dv.transpose('i','j','time')
            #
            try:
                %time dv.to_zarr(file_out, mode='w')
            except:
                print('Failure')
            dsize = getsize(file_out)
            print('face=%d / i=%d'%(face,i))
            print('   data is %.1fGB ' %(dsize/1e9))
        else:
            print('face=%d / i=%d - allready processed'%(face,i))

/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:314: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


KeyboardInterrupt: 

--- 

## standard data layout: (face, time, j, i) chunks = (1, 1, 4320, 4320)

In [11]:
V = ['Eta', 'SST', 'SSS', 'SSU', 'SSV']
V = ['SSU']

Nt = 1

#Nc = 432 # original choice
#Nc = 96 #
#Nc = 288
#Nc = 27 # very long scheduling

out_dir = scratch+'/mit_zarr/'

for v in V:
    #
    data_dir = root_data_dir+v+'/'
    iters, time = get_iters_time(v, data_dir, delta_t=25.)
    #
    p = 'C'
    if v is 'SSU':
        p = 'W'
    elif v is 'SSV':
        p = 'S'
    #
    ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
    print(ds)
    ds = ds.chunk({'face': 1})
    # tmp ...
    #ds = ds.isel(face=slice(1,3))
    #ds = ds.isel(face=1)
    # 
    #dv = ds[v].isel(time=slice(t,t+Nt))
    dv = ds[v]
    # should store grid data independantly in a single file
    dv = dv.drop(['XC','YC','Depth','rA']).to_dataset()        
    #
    #dv = dv.chunk({'face': 1, 'time': Nt, 'i': Nc, 'j': Nc})
    #dv = dv.chunk({'i': Nc, 'j': Nc}) # scheduler does not like this, i.e. takes a long time to do anything
    #
    file_out = out_dir+'/%s.zarr'%(v)
    try:
        print(dv)
        %time dv.to_zarr(file_out, mode='w')                    
    except:
        print('Failure')
    dsize = getsize(file_out)
    print('   data is %.1fGB ' %(dsize/1e9))

/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:314: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, i_g: 4320, j: 4320, j_g: 4320, k: 90, k_l: 90, k_p1: 91, k_u: 90, time: 8785)
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j_g      (j_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * k_u      (k_u) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k_l      (k_l) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k_p1     (k_p1) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
    XC       (face, j, i) >f4 dask.array<shape=(13, 4320, 4320), chunksize=(1, 4320, 4320)>
    YC       (face, j, i) >f4 dask.array<shape=(13, 4320, 4320), chunksize=(

distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)


<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, j: 4320, time: 8785)
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * time     (time) float64 5.702e+06 5.706e+06 5.71e+06 5.713e+06 5.717e+06 ...
    dtime    (time) datetime64[ns] dask.array<shape=(8785,), chunksize=(8785,)>
Data variables:
    SSU      (time, face, j, i) >f4 dask.array<shape=(8785, 13, 4320, 4320), chunksize=(1, 1, 4320, 4320)>


distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

CPU times: user 9min 20s, sys: 26.3 s, total: 9min 46s
Wall time: 16min 33s
   data is 4082.7GB 


---
## try to load standard lay out, rechunk and store right away

Does not work

In [7]:
#file_in = file_out
v = 'SSU'
file_in = scratch+'/mit_zarr/'+'/%s.zarr'%(v)
ds = xr.open_zarr(file_in)
print(ds)

<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, j: 4320, time: 8785)
Coordinates:
    dtime    (time) datetime64[ns] dask.array<shape=(8785,), chunksize=(8785,)>
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * time     (time) float64 5.702e+06 5.706e+06 5.71e+06 5.713e+06 5.717e+06 ...
Data variables:
    SSU      (time, face, j, i) float32 dask.array<shape=(8785, 13, 4320, 4320), chunksize=(1, 1, 4320, 4320)>


In [13]:
Nt = 24*10 # time chunks
#Nt = 0

Nc = 432 # original choice
#Nc = 96 #
#Nc = 27 # very long scheduling

out_dir = scratch+'/mit_zarr_t/'
#out_dir = datawork+'/mit_zarr_t/'
#out_dir = osi+'mit_zarr_t/'

#
lNt = len(ds.time) if Nt == 0 else Nt
#
#dv = ds[v].isel(time=slice(len(ds.time)//lNt *lNt))
dv = ds.isel(time=slice(len(ds.time)//lNt *lNt))
print(dv)
#
#dv = dv.chunk({'time': lNt})
#dv = dv.chunk({'time': lNt, 'i': Nc, 'j': Nc})
dv = dv.chunk({'i': Nc, 'j': Nc})
#dv = dv.chunk({'face': 1, 'time': lNt, 'i': Nc, 'j': Nc})
#
#dv = dv.transpose('face','i','j','time') # commented tmp, still leads to memory blow up
#
file_out = out_dir+'%s.zarr'%(v)
try:
    print(dv)
    %time dv.to_zarr(file_out, mode='w')
except:
    print('Failure')
dsize = getsize(file_out)
print('   data is %.1fGB ' %(dsize/1e9))


<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, j: 4320, time: 8640)
Coordinates:
    dtime    (time) datetime64[ns] dask.array<shape=(8640,), chunksize=(8640,)>
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * time     (time) float64 5.702e+06 5.706e+06 5.71e+06 5.713e+06 5.717e+06 ...
Data variables:
    SSU      (time, face, j, i) float32 dask.array<shape=(8640, 13, 4320, 4320), chunksize=(1, 1, 4320, 4320)>
<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, j: 4320, time: 8640)
Coordinates:
    dtime    (time) datetime64[ns] dask.array<shape=(8640,), chunksize=(8640,)>
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * time     (time) float64 5.702e

NotImplementedError: Specified zarr chunks (1, 1, 4320, 4320) would overlap multiple dask chunks ((1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

   data is 0.0GB 


In [ ]:
# can I rechunk along time and perform an operation with zarr data of chunk 1 initially?
# we should as it seems to be a IO sync/lock issue

# need to load data in memory to properly rechunk and store, this requires considering one face at a time


---

In [17]:
w = cluster.start_workers(10)

In [ ]:
client.restart()

In [14]:
# kill scheduler, workers
cluster.scheduler.close()
#cluster.stop_workers(cluster.jobs)

<Future pending cb=[_make_coroutine_wrapper.<locals>.wrapper.<locals>.<lambda>() at /home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/tornado/gen.py:336]>